In [1]:
code = '''
import click
from pathlib import Path
import pyperclip
import tiktoken
from turbo_docs.commands import readme as readme_module
from turbo_docs.commands import docs as docs_module
from turbo_docs.utils import directory, cli_options

def resolve_model(gpt3):
    if gpt3:
        model = "gpt-3.5-turbo-16k"
    else:
        model = "gpt-4"
        print(
            "Warning: This model is under limited beta access and is not available to all users."
        )
        print("Warning: GPT-4 api calls tend to be slower than other models.")

    print(f"Using model: {model}")
    return model

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string, allowed_special="all"))
    return num_tokens

@click.command()
@cli_options.copy
@cli_options.readme
@cli_options.gpt3
@cli_options.docs
@cli_options.narrative
def driver(
    copy: bool,
    readme: bool,
    gpt3: bool,
    docs: bool,
    narrative: str,
) -> None:
    """
    Pull text from all files in the current directory and apply the following commands:

    'turbo_docs --copy' copies the text to clipboard
        Useful for wokring with ChatGPT

    'turbo_docs --readme' generates a README.md file
        Useful for keeping documentation up to date

    'turbo_docs --gpt3' uses GPT-3.5-turbo-16k
        Useful if you don't have GPT-4 access

    'turbo_docs --docs' generates a docs for each file
        Useful for keeping documentation up to date
    """
    dir_text_dict = directory.get_repo_text_dict()

    if readme:
        if Path("README.md") in dir_text_dict:
            del dir_text_dict[Path("README.md")]
        dir_text_str = directory.convert_dict_to_string(dir_text_dict)
        model = resolve_model(gpt3)
        readme_module.readme(dir_text_str, model, narrative=narrative)
        print("Generated README.md")

    if docs:
        model = resolve_model(gpt3)
        docs_module.docs(dir_text_dict, model)
        print("Generated documentation")

    if copy:
        dir_text_str = directory.convert_dict_to_string(dir_text_dict)
        num_tokens = num_tokens_from_string(dir_text_str)
        pyperclip.copy(dir_text_str)
        print(f"Directory text copied to clipboard containing {num_tokens} tokens")
'''

In [12]:
from llm_blocks import blocks

In [13]:
template = """
Here is {filename} from {reponame}:
{code}

Can you convert this python code to pseudocode? Focus on the high-level logic, not the syntax.
"""

In [14]:
block = blocks.StreamBlock(template=template)
response_generator = block(filename="generate.py", reponame="turbo-docs", code=code)
block.display(response_generator)

Pseudocode for generate.py:

1. Import necessary libraries and modules.
2. Define a function named "resolve_model" that takes a boolean parameter "gpt3".
3. Inside the "resolve_model" function:
   a. Check the value of "gpt3".
   b. If "gpt3" is True, set the variable "model" to "gpt-3.5-turbo-16k".
   c. If "gpt3" is False, set the variable "model" to "gpt-4".
   d. Print a warning message if "gpt3" is False.
   e. Print the selected model.
   f. Return the value of "model".
4. Define a function named "num_tokens_from_string" that takes a string parameter "string" and an encoding name parameter "encoding_name".
5. Inside the "num_tokens_from_string" function:
   a. Get the encoding using the "encoding_name".
   b. Calculate the number of tokens in the "string" using the encoding.
   c. Return the number of tokens.
6. Define a function named "driver" that takes several boolean and string parameters.
7. Inside the "driver" function:
   a. Get the text from all files in the current direc

In [15]:
block = blocks.BatchBlock(template=template, model_name="gpt-3.5-turbo")
response = block(filename="generate.py", reponame="turbo-docs", code=code)
block.display(response)

{
  "id": "chatcmpl-7pykiJVmnlPkXdgrFPBa1mbi3QuPb",
  "object": "chat.completion",
  "created": 1692622616,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Pseudocode for generate.py:\n\n1. Import necessary libraries and modules.\n2. Define a function named \"resolve_model\" that takes a boolean parameter \"gpt3\".\n3. Inside the \"resolve_model\" function:\n   a. Check the value of \"gpt3\".\n   b. If \"gpt3\" is True, set the variable \"model\" to \"gpt-3.5-turbo-16k\".\n   c. If \"gpt3\" is False, set the variable \"model\" to \"gpt-4\".\n   d. Print a warning message if \"gpt3\" is False.\n   e. Print the selected model.\n   f. Return the value of \"model\".\n4. Define a function named \"num_tokens_from_string\" that takes a string parameter \"string\" and an encoding name parameter \"encoding_name\".\n5. Inside the \"num_tokens_from_string\" function:\n   a. Get the encoding for the specifi